In [0]:
import datetime
date_ = datetime.datetime.now().strftime("%Y-%m-%d")
silver_path = f"abfss://projectfiles@practicestorageacc0.dfs.core.windows.net/videostream/silver/{date_}/"
df_silver = spark.read.format("parquet").load(silver_path)
# df_silver.display()

In [0]:
from pyspark.sql.functions import date_trunc, col, hour

df_prepared = df_silver.withColumn("date", date_trunc("hour", col("created_at")))
# df_prepared.display()

In [0]:
from pyspark.sql.functions import max, countDistinct, count

df_gold_videoByHour = df_prepared.groupBy("video_id", "date")\
    .agg(
        max("viewers_count").alias("peak_views"),
        count("chat_message").alias("num_messages"),
        countDistinct("user_id").alias("unique_users")
    )

In [0]:
df_gold_userActivityByHour = df_prepared.groupBy("video_id", "user_id", "date")\
    .agg(
        count("event_type").alias("num_events"),
        count("chat_message").alias("num_messages")
    )
        
    

In [0]:
gold_path = f"abfss://projectfiles@practicestorageacc0.dfs.core.windows.net/videostream/gold/"
videoByHour_path = gold_path + "/videoByHour/" + date_ + "/"
userActivityByHour_path = gold_path + "/userActivityByHour/" + date_ + "/"

# Writing to Delta, to enable time travel, ACID operation functionalitites
df_gold_videoByHour.write\
    .mode("overwrite")\
    .format("delta")\
    .save(videoByHour_path)

df_gold_userActivityByHour.write\
    .mode("overwrite")\
    .format("delta")\
    .save(userActivityByHour_path)